# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Изучим общую информацию о таблице

In [1]:
import pandas as pd
from pymystem3 import Mystem

Выгрузим данную нам таблицу и сохраним его в переменной **credit_df**:

In [2]:
credit_df = pd.read_csv('/home/kseniya/Downloads/data.csv')

Получим первые 10 строк таблицы

In [3]:
credit_df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице **credit_df** с помощью метода ***info***:

In [4]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице *12 столбцов*, имеющих следующие типы данных:

    1) float ('days_employed', 'total_income');
    2) int ('children', 'dob_years', 'education_id', 'family_status_id', 'debt')
    3) object ('education', 'family_status', 'gender', 'income_type', 'purpose')

Подробно разберем, какую информацию содержат столбцы **credit_df**:

* *children* — количество детей в семье;

* *days_employed* — общий трудовой стаж в днях;

* *dob_years* — возраст клиента в годах;

* *education* — уровень образования клиента;

* *education_id* — идентификатор уровня образования;

* *family_status* — семейное положение;

* *family_status_id* — идентификатор семейного положения;

* *gender* — пол клиента;

* *income_type* — тип занятости;

* *debt* — имел ли задолженность по возврату кредитов;

* *total_income* — ежемесячный доход;

* *purpose* — цель получения кредита.
    
Количество значений в столбцах различается - значит в данных есть пропущенные значения. Есть проблема со столбцом *'days_employed'* - общий трудовой стаж в днях не может быть дробным и отлицательным числом. Нужна корректировка в столбце *'purpose'* - есть дублирующиеся по смыслу, но не по словам значения. В столбце *'education'* значения представлены в разных регистрах. 

### 1.1. Вывод

Мы выгрузили таблицу **credit_df** и с помощью метода ***info()*** рассмотрели общую информацию о ней. Всего в таблице *12 столбцов*. Табица состоит из *21525 строк*, однако в данных есть пропуски (*days_employed* и *total_income*). Пропуски в столбцах с подобной информацией возможны - например, студент может не иметь трудового стажа, также  мог иметь место человеческий фактор - сотрудник, вбивавший данные,  мог не внести их. В следующем шаге мы попробуем избывиться от пропущенных значений.

## Шаг 2. Предобработка данных

### 2.1. Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [5]:
credit_df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Начнем со столбца *'total_income'*. Пропуски свидетельствуют, что о некоторых кредитуемых предоставлена не вся информация. Причина может быть в том, что человек  безработный (в этом случае возникает вопрос, как банк мог одобрить ему кредит?). Нужно выявить причину пропусков.

Выясним, есть ли в столбце *'тип занятости' ('income_type')* значение "безработный". Для этого определим и подсчитаем все уникальные значения этого столбца.

In [6]:
credit_df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Искомое нами значение в столбце есть, но повторяется оно всего лишь два раза, а пропусков гораздо больше. Значит есть проблемы с записью данных.

Чтобы избавиться от пропусков, создадим цикл: для каждого типа занятости из столбца *'income_type'* найдем отсутствующие значения в столбце *'total_income'*, расчитаем медиану для каждого типа занятости и заменим отсутствующие значения. 

Создадим переменную **income_type_list**, в ней будет список уникальных значений столбца *'income_type'*.

In [7]:
income_type_list= credit_df['income_type'].unique()
income_type_list

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [8]:
for income_type_name in income_type_list:
    credit_df.loc[(credit_df['income_type'] == income_type_name) & (credit_df['total_income'].isna()), 'total_income'] \
    = credit_df.loc[(credit_df['income_type'] == income_type_name), 'total_income'].median()

Проверим, остались ли пропуски в столбце *'total_income'*:

In [9]:
credit_df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Хотя для нашего исследования столбец *days_employed* не пригодится, его пропуски мы тоже обработаем. Воспользуемся для этого более простым методом: заменим отсутствующие значения на *0*.

In [10]:
credit_df['days_employed'] = credit_df['days_employed'].fillna('0')

Еще раз проверяем таблицу на наличие пропусков.

In [11]:
credit_df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Мы выдвинули предположение, что пропуски в двух столбцах (*'days_employed'('общий трудовой стаж')* и *'total_income'('ежемесячный доход')*) возможны по причине того,что некоторые кредитуемые безработные. Однако, догадка не подтвердилась - безработных оказалось всего двое. Значит данные были введены некорректно.

### 2.2. Замена типа данных

Продолжим работу со столбцами *'days_employed'* и *'total_income'*. 

Поскольку столбец *'total_income'* включает в себя только числовые значения, его тип данных мы можем поменять с помощью метода **astype**.

In [12]:
credit_df['total_income'] = credit_df['total_income'].astype('int')

В столбец *'days-employed'* мы поместили строковые значения ("no_info"). Поэтому его тип данных мы поменяем с помощью метода **to_numeric** (мы можем задать его параметру *'errors'* действие "ignore" - он поменяет числа на целые, а строковые значения оставит такими, какие они есть).  

In [13]:
credit_df['days_employed'] = pd.to_numeric(credit_df['days_employed'])

Проверяем, изменились ли типы данных столбцов:

In [14]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Мы применили методы **astype** и **to_numeric** для изменения типа данных столбцов. Столбец *'days_employed'* теперь имеет тип данных object, но с ним в дальнейшем мы работать не будем, поэтому такой вариант нас устроит. А вот в столбце *'total_income'* тип данных поменялся на 'int' - с целыми числами нам будет гораздо удобнее производить расчеты.

### 2.3. Обработка дубликатов

Посмотрим на столбцы и исправим ошибки в них, если они есть.

Смотря на первые 10 строк таблицу, заметим,что в столбце *'education'* значения даны в разных регистрах. Убедимся в этом, выявив все уникальные значения этого столбца

In [15]:
credit_df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приведем все строки столбца к нижнему регистру

In [16]:
credit_df['education'] = credit_df['education'].str.lower()

Посмотрим на результат

In [17]:
credit_df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Посмотрим на столбец *'children'* - он пригодится нам для ответов на поставленные вопросы:

In [18]:
credit_df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Столбец включает в себя два аномальных значения - *20* и *-1*. Скорее всего, это опечатки - в значении *20* лишний 0, а в значении *-1* - "-". Произведем замену этих значений.

In [19]:
credit_df['children'] = credit_df['children'].replace(20, 2)

In [20]:
credit_df['children'] = credit_df['children'].replace(-1, 1)

In [21]:
credit_df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Проверим таблицу на наличие дубликатов:

In [22]:
credit_df.duplicated().sum()

71

Видим, что таблица содержит незначительное количество дубликатов (по сравнению с длиной всей таблицы), поэтому мы можем их удалить:

In [23]:
credit_df = credit_df.drop_duplicates().reset_index(drop=True)

Проверяем, остались ли дубликаты:

In [24]:
credit_df.duplicated().sum()

0

### Вывод

Возможная причина возникновения ошибок в столбцах *'education'* и *'children'* - человеческий фактор (например, если клиенты банка заполняли анкеты сами так, как считали нужным). Мы очистили таблицу от дубликатов - теперь перед нами более ясная картина.

### 2.4.Лемматизация

В столбце *'purpose' ('цель получения кредита')*  мы можем видеть, что на самом деле есть несколько основных целей, на которые люди берут у банка средства, но у одной и той же цели может быть несколько разных формулировок. Необходимо определить сколько целей в действительности.

Для этого нужно вычленить ключевые слова с помощью лемматизации. Для этого посмотрим на уникальные значения столбца *'purpose'*.

In [25]:
credit_df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Мы можем найти в этом столбце 5 слов, которые повторяются чаще всего:

* жилье;
* недвижимость;
* автомобиль;
* свадьба;
* образование

Именно эти слова будут нашими леммами.

Применим метод лемматизации к столбцу *'purpose'*. Значения поместим в отдельный столбец *'purpose_lem'*.

In [26]:

m = Mystem()
credit_df['purpose_lem'] = credit_df['purpose'].apply(m.lemmatize)

Installing mystem to /home/kseniya/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


Теперь нужно отделить конкретные цели в отдельный столбец - *'purpose_category'*. Для этого напишем функцию, которая будет искать значения по ячейкам. 
**Примечание**: пусть функция возвращает значение *'жилье'* как *'недвижимость'*. Такая групповка пригодится нам для более точного ответа на вопрос о том, как цели кредита влияют на его возврат в срок

In [27]:
def purpose_category(cell):
     
   
        if 'жилье' in cell:
            return 'недвижимость'
    
        if 'недвижимость' in cell:
            return 'недвижимость'
    
        if 'свадьба' in cell:
            return 'свадьба' 
    
        if 'образование' in cell:
            return 'образование'
    
        if 'автомобиль' in cell:
            return 'автомобиль'
        else:
            return 'другое'

Прибавим столбец *'purpose_category'* в таблицу. Применим функцию к столбцу *'purpose_lem'*. Выведем три столбца таблицы df - *'purpose'*, *'purpose_lem'* и *'purpose_category'*.

In [28]:
credit_df['purpose_category'] = credit_df['purpose_lem'].apply(purpose_category)
credit_df[['purpose','purpose_lem','purpose_category']]

,purpose,purpose_lem,purpose_category
0,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
...,...,...,...
21449,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
21450,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль
21451,недвижимость,"[недвижимость, \n]",недвижимость
21452,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",автомобиль


Проверим столбец *'purpose_category'* на уникальные значения и посчитаем их.

In [29]:
credit_df['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

### Вывод

В столбце *'purpose'* было много одинаковых по смыслу, но разных по форме значений. Это мешало увидеть реальное положение вещей - на какие цели больше всего берут кредит? С помощью лемматизации столбца *'purpose'* и создания функции, мы можем видеть, что целей всего 4 - **недвижимость, автомобиль** (*покупка, ремонт и др. манипуляции*) **, образование и свадьба**.

После того, как мы посчитали все уникальные значения столбца, можно увидеть, что подавляющее большинство клиентов банка берут кредит на недвижимость (покупку, ремонт). 

### 2.5.Категоризация данных

Вcе вопросы связаны со столбцом *'debt' ('имел ли задолженность по возврату кредитов')*. Поэтому сразу посмотрим и посчитаем все уникальные значения столбца.

In [31]:
credit_df['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

Видим, что добросовестных плательщиков гораздо больше. 

Приступим к *категоризации данных*.

Начнем с детей. Напишем функцию, которая будет учитывать нужные нам два столбца: *'children'* и *'debt'*. Работать функция будет в соответствии со следующей логикой: если дети есть (их **больше** 0) и у клиента были долги по выплатам, то пусть функция возвращает команду **'обратить внимание'**; во всех других случаях пусть возвращает значение **'другое'** .

In [32]:
def children_category(row):
    children = row['children']
    debt = row['debt']
    
    if children > 0:
        if debt == 1:
            return 'обратить внимание'
    
    return 'другое'

Добавим столбец *'children_category'* в таблицу и посмотрим статистику.

In [33]:
credit_df['children_category'] = credit_df.apply(children_category, axis=1)
credit_df['children_category'].value_counts()

другое               20776
обратить внимание      678
Name: children_category, dtype: int64


Чтобы понять ситуацию с зависимостью своевременных платежей по кредитам от семейного положения, нам понадобится столбец *'family_status'*.

Здесь (и в остальных случаях) мы будем пользоваться логической индексацией. Создаем столбец *'family_category'*, где будут все значения из столбца *'family_status'*, которые соответствуют значению *'1'* столбца *'debt'*

In [34]:
credit_df['family_category'] = credit_df[credit_df['debt'] == 1]['family_status']

Посмотрим на результаты:

In [35]:
credit_df['family_category'].value_counts()

женат / замужем          931
гражданский брак         388
Не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
Name: family_category, dtype: int64

Еще нас интересует, зависит ли наличие долгов по кредиту от уровня дохода. 
Применим к столбцу *'total_income'* функцию **describe()**. Она покажет нам описательную статистику по столбцу. 

In [36]:
credit_df['total_income'].describe().astype('int')

count      21454
mean      165319
std        98187
min        20667
25%       107623
50%       142594
75%       195820
max      2265604
Name: total_income, dtype: int64

Мы можем использовать  полученные числа как границы зарплатных категорий. Напишем функцию, которая поможет нам разделить значения на уровни доходов:
*  низкий
*  средний
*  высокий

In [37]:
def total_income_groups(total_income):
    low = 107798
    middle = 145017
    high = 195543
    
    if total_income >= high:
        return 'высокий доход'
    
    elif middle <= total_income < high:
        return 'средний доход'
    else:
        return 'низкий доход'

Добавим столбец *'total_income_groups'*, а затем с его создадим столбец *'total_income_category'*, в котором выведем значения *'total_income_groups'*, соответствующие *'1'* в столбце *'debt'*.

In [38]:
credit_df['total_income_groups'] = credit_df['total_income'].apply(total_income_groups)
credit_df['total_income_category'] = credit_df[credit_df['debt'] == 1]['total_income_groups']

Посчитаем все уникальные значения столбца *'total_income_category'*.

In [39]:
credit_df['total_income_category'].value_counts()

низкий доход     937
средний доход    418
высокий доход    386
Name: total_income_category, dtype: int64

Создадим словари для классификации данных по:

* уровню образования
* семейному положению
* основным целям кредита заёмщиков

In [40]:
education_dict = credit_df[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.sort_values('education_id')

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [42]:
family_dict = credit_df[['family_status', 'family_status_id']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict.sort_values('family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [43]:
purpose_dict = credit_df[['purpose_category']]
purpose_dict = purpose_dict.drop_duplicates('purpose_category')
purpose_dict

,purpose_category
0,недвижимость
1,автомобиль
3,образование
4,свадьба


Выведем значения *'purpose_category'*, соответствующие *'1'* в столбце *'debt'*.

In [44]:
credit_df['purpose_category_debt'] = credit_df[credit_df['debt'] == 1]['purpose_category']
credit_df['purpose_category_debt'].value_counts()

недвижимость    782
автомобиль      403
образование     370
свадьба         186
Name: purpose_category_debt, dtype: int64

### Вывод

Только в одном случае нам понадобилась функция - в случае с решением вопроса о зависимости своевременных выплат по кредиту от количества детей. В остальных же случаях нам больше подошла логическая индексация.

## Шаг 3. Ответим на вопросы

### 3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

Расчитаем долю клиентов с детьми, у которых были долги перед банком. 

Создадим переменные:

* *children_debt* ( в ней будет количество долговых случаев у клиентов с детьми)
* *with_children* ( в ней посчитаем общее количество клиентов с детьми)
* *without_children* ( в ней считаем кличество клиентов без детей)
* *all_clients* (в ней поситаем общее количество клиентов)

In [45]:
children_debt = credit_df['children_category'][credit_df['children_category'] == 'обратить внимание'].count()
children_debt

678

In [46]:
with_children = credit_df['children'][credit_df['children'] > 0].count()
with_children

7363

In [47]:
without_children = credit_df['children'][credit_df['children'] == 0].count()
without_children

14091

In [48]:
all_clients = credit_df['children'].count()
all_clients

21454

Расчитаем доли:

In [49]:
with_children_share =  with_children / all_clients
'Доля клиентов с детьми: {:.0%}'.format(with_children_share)  

'Доля клиентов с детьми: 34%'

In [50]:
without_children_share =  without_children / all_clients
'Доля клиентов, не имеющих детей: {:.0%}'.format(without_children_share)  

'Доля клиентов, не имеющих детей: 66%'

In [51]:
children_share = children_debt / with_children
'Доля должников среди клиентов с детьми: {:.0%}'.format(children_share)  

'Доля должников среди клиентов с детьми: 9%'

In [53]:
without_children_debt_share = children_debt / without_children
'Доля должников среди клиентов, не имеющих детей: {:.0%}'.format(without_children_debt_share)  

'Доля должников среди клиентов, не имеющих детей: 5%'

### Вывод

9 % - не такая большая доля от всех клиентов, у которых есть дети. При том условии, что всем известно - траты на детей достаточно большие, и есть клиенты, у которых не один ребенок. Но все же люди в основной массе возвращают деньги в срок, что может только радовать. 
Отвечаем на поставленный вопрос: зависимости между наличием детей и возвратом кредита в срок нет.

### 3.2.Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посчитаем количество значений **'женат / замужем'** и **'гражданский брак'** из столбца *'family_category'*.

In [54]:
family_category_married = credit_df['family_category'][credit_df['family_category'] == 'женат / замужем'].count()
family_category_civil = credit_df['family_category'][credit_df['family_category'] == 'гражданский брак'].count()
family_category_not_single = family_category_married + family_category_civil
family_category_not_single

1319

Теперь посчитаем те же самые значения из столбца *'family_status'*.

In [55]:
family_status_married = credit_df['family_status'][credit_df['family_status'] == 'женат / замужем'].count()
family_status_civil = credit_df['family_status'][credit_df['family_status'] == 'гражданский брак'].count()
family_status_not_single = family_status_married + family_status_civil
family_status_not_single

16490

Вычислим долю должников от всех женатых / замужних клиентов.

In [56]:
not_single_debt_share = family_category_not_single / family_status_not_single
'Доля должников среди клиентов, которые состоят в браке (и гражданском, и официальном): {:.0%}'.format(not_single_debt_share)  

'Доля должников среди клиентов, которые состоят в браке (и гражданском, и официальном): 8%'

Повторим те же самые расчеты для остальных "семейных" категорий.

In [57]:
family_category_not_married = credit_df['family_category'][credit_df['family_category'] == 'Не женат / не замужем'].count()
family_status_not_married = credit_df['family_status'][credit_df['family_status'] == 'Не женат / не замужем'].count()
not_married_debt_share = family_category_not_married / family_status_not_married
'Доля должников среди клиентов, не состоящих в браке: {:.0%}'.format(not_married_debt_share) 

'Доля должников среди клиентов, не состоящих в браке: 10%'

In [58]:
family_category_divorced = credit_df['family_category'][credit_df['family_category'] == 'в разводе'].count()
family_status_divorced = credit_df['family_status'][credit_df['family_status'] == 'в разводе'].count()
divorced_debt_share = family_category_divorced / family_status_divorced
'Доля должников среди разведенных клиентов: {:.0%}'.format(divorced_debt_share) 

'Доля должников среди разведенных клиентов: 7%'

In [59]:
family_category_widow = credit_df['family_category'][credit_df['family_category'] == 'вдовец / вдова'].count()
family_status_widow = credit_df['family_status'][credit_df['family_status'] == 'вдовец / вдова'].count()
widow_debt_share = family_category_widow / family_status_widow
'Доля должников среди овдовевших клиентов: {:.0%}'.format(widow_debt_share) 

'Доля должников среди овдовевших клиентов: 7%'

### Вывод

Мы видим из наших расчетов, что в общем должники в своих группах составляют не такой уж большой процент. Но самое интересное,что самый большой процент составляют клиенты, не состоящие в браке - 10 %. 

Отвечаем на поставленный вопрос - нет, зависимости между семейным положением и возвратом кредита в срок нет.

### 3.3.Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим сводную таблицу. Названиями срок будут значения столбца *'total_income_groups'*, а в столбце *'debt'* будут значения, по которым мы хотим увидеть таблицу, т.е. количество должников. Применим к значениям функцию **count**, чтобы увидеть, в какой группе должников больше и дать ответ на поставленный вопрос. 

In [60]:
debt_data = credit_df.pivot_table(index='total_income_category', values='debt', aggfunc='count')
debt_data

,debt
total_income_category,
высокий доход,386
низкий доход,937
средний доход,418


Теперь посмотрим, как влияет уровень дохода и наличие детей на количество долговых случаев:

In [61]:
debt_income_data = credit_df.pivot_table(index=['children_category','total_income_category'], values='debt', aggfunc='count')
debt_income_data

debt
children_category total_income_category      
другое            высокий доход           237
                  низкий доход            576
                  средний доход           250
обратить внимание высокий доход           149
                  низкий доход            361
                  средний доход           168

Так же воспользуемся сводной таблицей, чтобы посмотреть, как влияет уровень дохода и семейное положение на количество долговых случаев:

In [62]:
debt_income_data1 = credit_df.pivot_table(index=['family_category','total_income_category'], values='debt', aggfunc='count')
debt_income_data1

debt
family_category       total_income_category      
Не женат / не замужем высокий доход            66
                      низкий доход            135
                      средний доход            73
в разводе             высокий доход            19
                      низкий доход             48
                      средний доход            18
вдовец / вдова        высокий доход            11
                      низкий доход             35
                      средний доход            17
гражданский брак      высокий доход            72
                      низкий доход            218
                      средний доход            98
женат / замужем       высокий доход           218
                      низкий доход            501
                      средний доход           212

### Вывод

Самый большой показатель должников у группы с низким уровнем дохода. Этот показатель ожидаем.

Но, как ни странно, клиенты с детьми более ответственны (что еще раз подтверждает наше предположение об отсутствии зависимости просрочек по кредитам от наличия детей).

А вот у клиентов, состоящих в официальном браке, большие показатели во всех трех категориях дохода. Но превалирует, конечно, низкий.

Ответ на вопрос - да, зависимость есть. Чем ниже доход, тем больше вероятность, что клиент будет иметь просроченные платежи по кредиту.

### 3.4.Как разные цели кредита влияют на его возврат в срок?

Посчитаем количество клиентов, взявших кредиты на каждую категорию:

In [63]:
all_clients_car = credit_df['purpose_category'][credit_df['purpose_category'] == 'автомобиль'].count()
all_clients_car

4306

In [64]:
all_clients_realty = credit_df['purpose_category'][credit_df['purpose_category'] == 'недвижимость'].count()
all_clients_realty

10811

In [65]:
all_clients_education = credit_df['purpose_category'][credit_df['purpose_category'] == 'образование'].count()
all_clients_education

4013

In [66]:
all_clients_wed = credit_df['purpose_category'][credit_df['purpose_category'] == 'свадьба'].count()
all_clients_wed

2324

Отобразим в сводной таблице количество должников по каждой категории:

In [68]:
purpose_debt_data = credit_df.pivot_table(index='purpose_category_debt', values='debt', aggfunc='count')
purpose_debt_data

,debt
purpose_category_debt,
автомобиль,403
недвижимость,782
образование,370
свадьба,186


Отобразим долю должников для каждой категории:

In [69]:
purpose_debt = credit_df.pivot_table(index='purpose_category', values='debt', aggfunc='mean')
purpose_debt

,debt
purpose_category,
автомобиль,0.093590
недвижимость,0.072334
образование,0.092200
свадьба,0.080034


### Вывод

Самый большой процент должников среди тех, кто берет кредиты на манипуляции с автомобилем и на образование - по 9%. 

Самый маленький процент должников среди тех,кто берет кредит на недвижимость - 7%.

## Шаг 4. Общий вывод

Нам необходимо было выяснить, влияет ли семейное положение, наличие детей на своевременные выплаты по кредитам. 

Исходя из расчетов: клиенты в детьми, как ни парадоксально, более ответственно подходят к выплатам - среди них процент должников весьма невелик.

А вот зависимость от семейного положения есть - женатые/замужние клиенты дали большие показатели среди должников,  не зависимо от уровня дохода(хотя у низкого дохода все равно самая большая цифра - 501 случай).

На своевренные выплаты влияет уровень дохода и цели кредита.

Самое большое количество должников у группы клиентов с низким уровнем дохода (931). 

Самый большой процентаж должников (по 9%) среди клиентов, которые берут кредит на образование и автомобиль.

Это вполне логично: скорее всего кредит на образование берут молодые люди. Даже если они работают, доход у них небольшой, так как опыт работы либо минимальный, либо его нет совсем. 

Как ни парадоксально, самый маленький процент должников среди тех, кто берет кредит на недвижимость - 7%. И хотя ежемесячные выплаты на такие цели достаточно велики, люди более ответственно подходят к таким выплатам, чтобы не потерять недвижимость. 

Гипотеза о зависимости наличия долгов от наличия детей не подтвердилась, а от семейного статуса, целей кредита и уровня дохода - наоборот. 